# 04. State Analysis (주별 분석)

Critical Ratios로 각 State의 유의미한 성격 특징 도출

In [ ]:
%pip install pandas numpy matplotlib seaborn -q

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
print(f'작업 폴더: {os.getcwd()}')

## 1. 데이터 준비

In [ ]:
# 데이터 로드 및 병합
data = pd.read_csv('data/raw/sapa_data.csv')
scores = pd.read_csv('data/processed/sapa_scores.csv')

df = scores.merge(data[['RID', 'state']], on='RID', how='left')
print(f"전체: {len(df):,}명")
print(f"\n=== State별 분포 ===")
print(df['state'].value_counts())

In [ ]:
# "other" 제외, 9개 주만 분석
states_9 = ['California', 'Florida', 'Illinois', 'Michigan', 
            'New York', 'Pennsylvania', 'Texas', 'Virginia', 'Washington']

df_states = df[df['state'].isin(states_9)].copy()
scales = ['NEO_O', 'NEO_C', 'NEO_E', 'NEO_A', 'NEO_N', 'Ideology', 'Honesty_Humility']

print(f"\n분석 대상: {len(df_states):,}명 (9개 주)")
print(f"\n=== 9개 주 표본 ===")
for state in states_9:
    n = len(df_states[df_states['state'] == state])
    print(f"  {state}: {n:,}명")

## 2. State별 기술통계

In [ ]:
# State × 척도별 N, Mean, SE 계산
stats_list = []

for state in states_9:
    state_data = df_states[df_states['state'] == state]
    for scale in scales:
        values = state_data[scale].dropna()
        if len(values) > 0:
            stats_list.append({
                'state': state,
                'scale': scale,
                'n': len(values),
                'mean': values.mean(),
                'sd': values.std(),
                'se': values.std() / np.sqrt(len(values))
            })

stats_df = pd.DataFrame(stats_list)
print(f"기술통계 계산 완료: {len(stats_df)}개 (9 states × 7 scales)")

## 3. Critical Ratios 계산

CR = (State Mean - Grand Mean) / SE

|CR| > 3.0 → 유의미 (p < .003)

In [ ]:
# Grand Mean (9개 주 전체)
grand_means = {scale: df_states[scale].mean() for scale in scales}

print("=== Grand Means ===")
for scale, gm in grand_means.items():
    print(f"  {scale}: {gm:.3f}")

In [ ]:
# Critical Ratios 계산
stats_df['grand_mean'] = stats_df['scale'].map(grand_means)
stats_df['cr'] = (stats_df['mean'] - stats_df['grand_mean']) / stats_df['se']
stats_df['significant'] = stats_df['cr'].abs() > 3.0

print("Critical Ratios 계산 완료")

In [ ]:
# 유의미한 특징 출력
sig = stats_df[stats_df['significant']].sort_values('cr', key=abs, ascending=False)

print(f"=== 유의미한 특징 (|CR| > 3.0) ===")
print(f"총 {len(sig)}개 발견\n")

for _, row in sig.iterrows():
    direction = "높음 ↑" if row['cr'] > 0 else "낮음 ↓"
    print(f"{row['state']:12} | {row['scale']:18} | CR={row['cr']:+.2f} | {direction}")

## 4. 시각화

In [ ]:
# CR 히트맵
cr_pivot = stats_df.pivot(index='state', columns='scale', values='cr')[scales]

fig, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(cr_pivot, annot=True, fmt='.1f', cmap='RdBu_r', center=0,
            linewidths=0.5, vmin=-6, vmax=6, cbar_kws={'label': 'Critical Ratio'})
ax.set_title('State별 성격 특성 Critical Ratios (|CR| > 3.0 = 유의미)')
plt.tight_layout()
plt.savefig('reports/state_cr_heatmap.png', dpi=150)
print('저장: reports/state_cr_heatmap.png')
plt.show()

## 5. State 성격 프로필 요약

In [ ]:
print("=== State 성격 프로필 요약 ===")
print("(|CR| > 3.0인 특성만)\n")

for state in states_9:
    state_sig = sig[sig['state'] == state]
    if len(state_sig) > 0:
        traits = [f"{row['scale']}{'↑' if row['cr'] > 0 else '↓'}" for _, row in state_sig.iterrows()]
        print(f"{state}: {', '.join(traits)}")
    else:
        print(f"{state}: (유의미한 특징 없음)")

In [ ]:
print("\n=== 분석 완료 ===")
print(f"분석 대상: 9개 주, {len(df_states):,}명")
print(f"유의미한 특징: {len(sig)}개")
print(f"생성 파일: reports/state_cr_heatmap.png")